In [84]:
import cv2
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal as norm
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [87]:
file_train_b_csv=r'/home/jacobo15defrutos/AVS9/5-Pruebas/train.csv'
df_train= pd.read_csv(file_train_b_csv)
labels_train= df_train['Label']
df_train

,Name,Label
0,739156_OD_SUP_SURV.jpg_aug.png,2
1,465557_OD_INF_SANO.jpg,0
2,9108332_OD_TEM_SURV.jpg_aug.png,2
3,331281_OD_SUP_CMV.jpg,1
4,9455268_OD_SUP_CMV.jpg,1
...,...,...
254,331281_OI_INF_SANO.jpg,0
255,649387_OI_NAS_SURV.jpg,2
256,9301219_OI_SUP_CMV.jpg_aug.png,1
257,2581150_OI_NAS_SANO.jpg,0


In [88]:
train_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_train['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img=img.ravel()
    train_list.append(img)

[ WARN:0@22975.784] global /croot/opencv-suite_1691620365762/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images739156_OD_SUP_SURV.jpg_aug.png'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /croot/opencv-suite_1691620365762/work/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
train= np.array(train_list)
train.shape

In [ ]:
file_test_b_csv=r'/home/jacobo15defrutos/AVS9/4-Red_Siamesa/Iris_test_basic_seg_list.csv'
df_test= pd.read_csv(file_test_b_csv)
labels_test= df_test['Label']
test_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/Data/saved_seg_class_images/test/'
for name in df_test['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img_dense = img.ravel()
    test_list.append(img_dense)
test= np.array(test_list)
test.shape

In [ ]:
clf = SVC(gamma='scale',decision_function_shape='ovo',kernel='rbf')
clf.fit(train,labels_train)

In [ ]:
pred=clf.predict(test)
from sklearn.metrics import accuracy_score
acc=accuracy_score(labels_test,pred)
acc

In [ ]:
file_val_b_csv=r'/home/jacobo15defrutos/AVS9/4-Red_Siamesa/Iris_val_basic_seg_list.csv'
df_val= pd.read_csv(file_val_b_csv)
labels_val= df_val['Label']
val_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/Data/saved_seg_class_images/val/'
for name in df_val['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img_dense = img.ravel()
    val_list.append(img_dense)
val= np.array(val_list)
val.shape
pred_val=clf.predict(val)
acc_v=accuracy_score(labels_val,pred_val)
acc_v

### Reducimos dimensionalidad para ver en 2 dimensiones

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda=LDA(n_components=2)
lda.fit(train,labels_train)

In [ ]:
train_SANO_list=[]
train_CMV_list=[]
train_SURV_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/Data/saved_seg_class_images/train_total/'
for name in df_train['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img=img.ravel()
    if 'SANO' in name:
        train_SANO_list.append(img)
    elif 'CMV' in name:
        train_CMV_list.append(img)
    elif 'SURV' in name:
        train_SURV_list.append(img)
train_SANO= np.array(train_SANO_list)
train_CMV= np.array(train_CMV_list)
train_SURV= np.array(train_SURV_list)
train_CMV.shape

In [ ]:
x_sano=lda.transform(train_SANO)
x_cmv=lda.transform(train_CMV)
x_surv=lda.transform(train_SURV)
figsize=(10,6)
fig ,ax = plt.subplots(figsize=figsize)
ax.grid()
ax.scatter(x_sano[:,0],x_sano[:,1],c='b',marker=".",label="Class SANO")
ax.scatter(x_cmv[:,0],x_cmv[:,1],c='r',marker=".",label="Class CMV")
ax.scatter(x_surv[:,0],x_surv[:,1],c='g',marker=".",label="Class SURV")
ax.legend()